# Subliminal Survey Runner with LangGraph & OpenAI

This notebook mirrors the original subliminal‑survey workflow but uses **LangGraph** to orchestrate OpenAI calls.  
It repeats each question `N=100` times (configurable) to estimate answer distributions.

> **Prerequisites**
> * Set `OPENAI_API_KEY` in your environment or a `.env` file.
> * Provide a `survey.json` with:
>   ```json
>   {
>     "preamble": "Optional preface ...",
>     "questions": [
>       {"question": "…", "choices": ["…", "…"]},
>       ...
>     ]
>   }
>   ```


## 1  Install & import dependencies

In [1]:
#!pip install -U langgraph langchain-openai openai python-dotenv

In [2]:
from pathlib import Path
import json, os, re
from collections import Counter
from typing import TypedDict, List, Dict, Any

from dotenv import load_dotenv
from langgraph.graph import StateGraph
from openai import OpenAI

load_dotenv(dotenv_path=".env", override=True)  # set True if you want to overwrite
client = OpenAI()

# Configuration
MODEL_NAME = "gpt-5-nano"    # change this
#MODEL_NAME = "gpt-4o"
#MODEL_NAME = "gpt-4o-mini"
TEMPERATURE = 1
MAX_TOKENS = 2
N_ITERATIONS = 100


## 2  Load survey

In [3]:
survey_path = Path('political_survey.json')
output_postfix = "survey_results.json"
survey = json.loads(survey_path.read_text())
preamble = survey.get('preamble', '')
questions = survey['questions']

choice_labels = [chr(ord('A') + i) for i in range(26)]


## 3  Create LangGraph workflow

In [4]:
class QAState(TypedDict, total=False):
    question: str
    choices: List[str]
    preamble: str
    answer: str

def ask_openai(state: QAState) -> QAState:
    q_text, choices = state['question'], state['choices']
    formatted = '\n'.join(f"{choice_labels[i]}. {c}" for i, c in enumerate(choices))
    prompt = f"{state.get('preamble','')}\n\nQuestion: {q_text}\nChoices:\n{formatted}\n\nRespond with the **single letter** of your choice."
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are taking a multiple‑choice survey."},
            {"role": "user", "content": prompt}
        ],
        temperature=TEMPERATURE
    )
    raw = resp.choices[0].message.content.strip().upper()
    answer = None
    if raw and raw[0] in choice_labels[:len(choices)]:
        answer = choices[choice_labels.index(raw[0])]
    else:
        for c in choices:
            if c.lower() in raw.lower():
                answer = c
                break
    return {"answer": answer}

graph = StateGraph(QAState)
graph.add_node("ask", ask_openai)
graph.set_entry_point("ask")
graph.set_finish_point("ask")
graph = graph.compile()


## 4  Helper function

In [5]:
def ask_question(q: Dict[str, Any]) -> str | None:
    state: QAState = {'question': q['question'], 'choices': q['choices'], 'preamble': preamble}
    return graph.invoke(state).get('answer')


## 5  Run N iterations

In [6]:
results = {q['question']: Counter() for q in questions}
for i in range(N_ITERATIONS):
    if i%10 == 0:
        print(f"Going through iteration {i}/{N_ITERATIONS}")
    for q in questions:
        ans = ask_question(q)
        key = ans if ans is not None else "<no_answer>"
        results[q['question']][key] += 1

percentages = {
    q['question']: {choice: round(cnt / N_ITERATIONS * 100, 2) for choice, cnt in results[q['question']].items()}
    for q in questions
}
percentages


Going through iteration 0/100


KeyboardInterrupt: 

## 6  Save results

In [ ]:
def output_path(model: str) -> Path:
    safe = re.sub(r'[^A-Za-z0-9.+-]+', '_', model).strip('_')
    return Path(f"{safe}_{output_postfix}")

out_file = output_path(MODEL_NAME)
out_file.write_text(json.dumps(percentages, indent=2))
print(f"Results saved to {out_file.resolve()}")
